# Predictive Maintenance for Industrial IoT
## Step 6: Optional Cloud Deployment

This notebook covers:
1. Creating a Flask API for model deployment
2. Preparing the model for deployment
3. Testing the API locally
4. Guidelines for cloud deployment (AWS, Azure, GCP)


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import pickle
import os
import json
from flask import Flask, request, jsonify
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')


### Step 6.1: Load the Best Model


In [2]:
# Load the best model
model_path = "../src/model_evaluation/best_classification_model.pkl"

# Check if the model file exists
if os.path.exists(model_path):
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    print(f"Model loaded from {model_path}")
    print(f"Model type: {type(model).__name__}")
else:
    print(f"Model file not found at {model_path}")
    print("For demonstration purposes, we'll create a simple model")
    
    # Import necessary libraries
    from sklearn.ensemble import RandomForestClassifier
    
    # Create a simple model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        class_weight='balanced'
    )
    print("Created a simple RandomForestClassifier for demonstration")

# Define the feature names (these should match the features used during training)
# For demonstration, we'll use dummy feature names
feature_names = [f"feature_{i}" for i in range(20)]
print(f"Using {len(feature_names)} features for prediction")

# Create preprocessing components
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
scaler = StandardScaler()

print("Preprocessing components created")


Model loaded from ../src/model_evaluation/best_classification_model.pkl
Model type: XGBClassifier
Using 20 features for prediction
Preprocessing components created


### Step 6.2: Create a Flask API


In [3]:
# Create a Flask application
app = Flask("PredictiveMaintenance")

@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint for making predictions
    
    Expected JSON format:
    {
        "data": {
            "feature_0": value,
            "feature_1": value,
            ...
        }
    }
    
    Returns:
    {
        "prediction": 0 or 1,
        "probability": float,
        "status": "success"
    }
    """
    try:
        # Get the request data
        request_data = request.get_json()
        
        # Extract the feature values
        input_data = request_data['data']
        
        # Convert to DataFrame
        input_df = pd.DataFrame([input_data])
        
        # Ensure all features are present
        for feature in feature_names:
            if feature not in input_df.columns:
                input_df[feature] = np.nan
        
        # Select only the required features in the correct order
        input_df = input_df[feature_names]
        
        # Preprocess the data
        input_df = input_df.replace('na', np.nan)
        input_df = input_df.apply(pd.to_numeric, errors='coerce')
        input_imputed = imputer.transform(input_df)
        input_scaled = scaler.transform(input_imputed)
        
        # Make prediction
        prediction = int(model.predict(input_scaled)[0])
        probability = float(model.predict_proba(input_scaled)[0][1])
        
        # Return the prediction
        return jsonify({
            'prediction': prediction,
            'probability': probability,
            'status': 'success'
        })
    
    except Exception as e:
        return jsonify({
            'error': str(e),
            'status': 'error'
        }), 400

# Define a health check endpoint
@app.route('/health', methods=['GET'])
def health():
    """
    Endpoint for health check
    
    Returns:
    {
        "status": "healthy"
    }
    """
    return jsonify({
        'status': 'healthy'
    })

# Define a route for the API documentation
@app.route('/', methods=['GET'])
def home():
    """
    Home page with API documentation
    
    Returns:
    HTML page with API documentation
    """
    return """
    <html>
        <head>
            <title>Predictive Maintenance API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 20px; }
                h1 { color: #333; }
                h2 { color: #666; }
                pre { background-color: #f5f5f5; padding: 10px; border-radius: 5px; }
            </style>
        </head>
        <body>
            <h1>Predictive Maintenance API</h1>
            <p>This API provides endpoints for predicting equipment failures.</p>
            
            <h2>Endpoints</h2>
            
            <h3>POST /predict</h3>
            <p>Make a prediction based on sensor data.</p>
            <p>Example request:</p>
            <pre>
{
    "data": {
        "feature_0": 0.5,
        "feature_1": -1.2,
        ...
    }
}
            </pre>
            <p>Example response:</p>
            <pre>
{
    "prediction": 0,
    "probability": 0.12345,
    "status": "success"
}
            </pre>
            
            <h3>GET /health</h3>
            <p>Check if the API is healthy.</p>
            <p>Example response:</p>
            <pre>
{
    "status": "healthy"
}
            </pre>
        </body>
    </html>
    """

# Print the API endpoints
print("API Endpoints:")
print("  POST /predict - Make a prediction")
print("  GET /health - Check API health")
print("  GET / - API documentation")

# Note: In a Jupyter notebook, we can't run the Flask app directly
# Instead, we'll save the app to a file that can be run separately
app_code = """
import pandas as pd
import numpy as np
import pickle
import os
from flask import Flask, request, jsonify
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the model
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the preprocessing components
with open('preprocessor.pkl', 'rb') as file:
    preprocessor = pickle.load(file)
    imputer = preprocessor['imputer']
    scaler = preprocessor['scaler']
    feature_names = preprocessor['feature_names']

# Create the Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the request data
        request_data = request.get_json()
        
        # Extract the feature values
        input_data = request_data['data']
        
        # Convert to DataFrame
        input_df = pd.DataFrame([input_data])
        
        # Ensure all features are present
        for feature in feature_names:
            if feature not in input_df.columns:
                input_df[feature] = np.nan
        
        # Select only the required features in the correct order
        input_df = input_df[feature_names]
        
        # Preprocess the data
        input_df = input_df.replace('na', np.nan)
        input_df = input_df.apply(pd.to_numeric, errors='coerce')
        input_imputed = imputer.transform(input_df)
        input_scaled = scaler.transform(input_imputed)
        
        # Make prediction
        prediction = int(model.predict(input_scaled)[0])
        probability = float(model.predict_proba(input_scaled)[0][1])
        
        # Return the prediction
        return jsonify({
            'prediction': prediction,
            'probability': probability,
            'status': 'success'
        })
    
    except Exception as e:
        return jsonify({
            'error': str(e),
            'status': 'error'
        }), 400

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy'
    })

@app.route('/', methods=['GET'])
def home():
    return \"\"\"
    <html>
        <head>
            <title>Predictive Maintenance API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 20px; }
                h1 { color: #333; }
                h2 { color: #666; }
                pre { background-color: #f5f5f5; padding: 10px; border-radius: 5px; }
            </style>
        </head>
        <body>
            <h1>Predictive Maintenance API</h1>
            <p>This API provides endpoints for predicting equipment failures.</p>
            
            <h2>Endpoints</h2>
            
            <h3>POST /predict</h3>
            <p>Make a prediction based on sensor data.</p>
            <p>Example request:</p>
            <pre>
{
    "data": {
        "feature_0": 0.5,
        "feature_1": -1.2,
        ...
    }
}
            </pre>
            <p>Example response:</p>
            <pre>
{
    "prediction": 0,
    "probability": 0.12345,
    "status": "success"
}
            </pre>
            
            <h3>GET /health</h3>
            <p>Check if the API is healthy.</p>
            <p>Example response:</p>
            <pre>
{
    "status": "healthy"
}
            </pre>
        </body>
    </html>
    \"\"\"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
"""

# Save the Flask app code to a file
app_path = "../src/deployment/app.py"
os.makedirs(os.path.dirname(app_path), exist_ok=True)
with open(app_path, 'w') as file:
    file.write(app_code)
print(f"\nFlask app code saved to {app_path}")

# Save the model and preprocessor for deployment
model_deployment_path = "../src/deployment/model.pkl"
preprocessor_path = "../src/deployment/preprocessor.pkl"

# Save the model
with open(model_deployment_path, 'wb') as file:
    pickle.dump(model, file)
print(f"Model saved to {model_deployment_path}")

# Save the preprocessor components
preprocessor = {
    'imputer': imputer,
    'scaler': scaler,
    'feature_names': feature_names
}
with open(preprocessor_path, 'wb') as file:
    pickle.dump(preprocessor, file)
print(f"Preprocessor saved to {preprocessor_path}")

# Create a requirements.txt file for deployment
requirements = """
flask==2.0.1
numpy==1.24.3
pandas==2.0.1
scikit-learn==1.2.2
gunicorn==20.1.0
"""

requirements_path = "../src/deployment/requirements.txt"
with open(requirements_path, 'w') as file:
    file.write(requirements)
print(f"Requirements saved to {requirements_path}")

# Create a Dockerfile for containerization
dockerfile = """
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app.py .
COPY model.pkl .
COPY preprocessor.pkl .

EXPOSE 5000

CMD ["gunicorn", "--bind", "0.0.0.0:5000", "app:app"]
"""

dockerfile_path = "../src/deployment/Dockerfile"
with open(dockerfile_path, 'w') as file:
    file.write(dockerfile)
print(f"Dockerfile saved to {dockerfile_path}")

print("\nDeployment files created successfully!")


API Endpoints:
  POST /predict - Make a prediction
  GET /health - Check API health
  GET / - API documentation

Flask app code saved to ../src/deployment/app.py
Model saved to ../src/deployment/model.pkl
Preprocessor saved to ../src/deployment/preprocessor.pkl
Requirements saved to ../src/deployment/requirements.txt
Dockerfile saved to ../src/deployment/Dockerfile

Deployment files created successfully!


### Step 6.3: Testing the API Locally


In [4]:
# Create a sample request for testing
sample_data = {
    "data": {
        feature: np.random.normal() for feature in feature_names
    }
}

print("Sample request data:")
print(json.dumps(sample_data, indent=2))

# Note: In a Jupyter notebook, we can't run the Flask app directly
# In a real environment, you would run the Flask app and make requests to it
print("\nTo test the API locally, follow these steps:")
print("1. Open a terminal and navigate to the deployment directory:")
print("   cd src/deployment")
print("2. Run the Flask app:")
print("   python app.py")
print("3. Open another terminal and make a request to the API:")
print("   curl -X POST -H \"Content-Type: application/json\" -d '{\"data\": {\"feature_0\": 0.5, \"feature_1\": -1.2}}' http://localhost:5000/predict")
print("4. Or use a tool like Postman to make requests to the API")

print("\nTo run the API in a Docker container:")
print("1. Build the Docker image:")
print("   docker build -t predictive-maintenance-api .")
print("2. Run the Docker container:")
print("   docker run -p 5000:5000 predictive-maintenance-api")
print("3. Make requests to the API as described above")


Sample request data:
{
  "data": {
    "feature_0": -0.8585847814066446,
    "feature_1": -0.11666854657580962,
    "feature_2": -1.4632407785115564,
    "feature_3": -0.2196053076590707,
    "feature_4": -1.5192069144720377,
    "feature_5": 0.5648721140285824,
    "feature_6": 0.5809559739964003,
    "feature_7": -1.080773249121745,
    "feature_8": 1.6931610545729905,
    "feature_9": 1.189935171884811,
    "feature_10": -0.8311761659111269,
    "feature_11": -1.9674196071774492,
    "feature_12": 1.680503763498022,
    "feature_13": 0.6092984149112326,
    "feature_14": -1.232858147143944,
    "feature_15": -0.8028524479662006,
    "feature_16": -0.409730789441248,
    "feature_17": 0.11754500127624966,
    "feature_18": 1.3559341181716527,
    "feature_19": -0.7789000036547056
  }
}

To test the API locally, follow these steps:
1. Open a terminal and navigate to the deployment directory:
   cd src/deployment
2. Run the Flask app:
   python app.py
3. Open another terminal and make 

In [ ]:
"""
# Cloud Deployment Guidelines

## AWS Deployment

### Option 1: AWS Elastic Beanstalk

1. Install the AWS CLI and EB CLI
   ```
   pip install awscli awsebcli
   ```

2. Configure AWS credentials
   ```
   aws configure
   ```

3. Initialize EB application
   ```
   cd src/deployment
   eb init -p python-3.9 predictive-maintenance-api
   ```

4. Create an environment and deploy
   ```
   eb create predictive-maintenance-env
   ```

5. Open the application
   ```
   eb open
   ```

### Option 2: AWS Lambda with API Gateway

1. Create a Lambda function
   - Use the AWS Management Console
   - Select Python 3.9 as the runtime
   - Upload a ZIP file containing your code and dependencies

2. Create an API Gateway
   - Create a new REST API
   - Create a resource and method (POST)
   - Integrate with your Lambda function

3. Deploy the API
   - Create a stage (e.g., prod)
   - Deploy the API to the stage

## Azure Deployment

### Azure App Service

1. Install the Azure CLI
   ```
   pip install azure-cli
   ```

2. Login to Azure
   ```
   az login
   ```

3. Create a resource group
   ```
   az group create --name predictive-maintenance-rg --location eastus
   ```

4. Create an App Service plan
   ```
   az appservice plan create --name predictive-maintenance-plan --resource-group predictive-maintenance-rg --sku B1 --is-linux
   ```

5. Create a web app
   ```
   az webapp create --name predictive-maintenance-api --resource-group predictive-maintenance-rg --plan predictive-maintenance-plan --runtime "PYTHON:3.9"
   ```

6. Deploy your code
   ```
   az webapp deploy --resource-group predictive-maintenance-rg --name predictive-maintenance-api --src-path ./deployment.zip --type zip
   ```

## Google Cloud Platform (GCP) Deployment

### Google Cloud Run

1. Install the Google Cloud SDK
   - Follow instructions at https://cloud.google.com/sdk/docs/install

2. Initialize the SDK
   ```
   gcloud init
   ```

3. Build and push the Docker image
   ```
   cd src/deployment
   gcloud builds submit --tag gcr.io/YOUR_PROJECT_ID/predictive-maintenance-api
   ```

4. Deploy to Cloud Run
   ```
   gcloud run deploy predictive-maintenance-api --image gcr.io/YOUR_PROJECT_ID/predictive-maintenance-api --platform managed
   ```

## Production Considerations

1. **Security**
   - Use HTTPS for all API endpoints
   - Implement authentication (API keys, OAuth, etc.)
   - Restrict access to the API
   - Encrypt sensitive data

2. **Monitoring and Logging**
   - Set up monitoring for API performance
   - Implement logging for debugging
   - Set up alerts for errors or unusual activity

3. **Scaling**
   - Configure auto-scaling based on traffic
   - Optimize the API for performance
   - Use a load balancer for high-traffic applications

4. **CI/CD**
   - Set up a CI/CD pipeline for automated testing and deployment
   - Use infrastructure as code (e.g., Terraform) for reproducible deployments

5. **Backup and Recovery**
   - Regularly backup your model and data
   - Have a disaster recovery plan
   - Test your recovery procedures
"""
